<a href="https://colab.research.google.com/github/dschloe/gcp_tutorial/blob/master/03_ML/BigQuery_with_Random_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BigQuery with Random Sampling
- 랜덤 샘플링을 해봅니다. 

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [8]:
%load_ext google.colab.data_table

In [24]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd
from pandas.io import gbq

project_id = 'evanproject-311200'
sql = '''
SELECT
  date, 
  airline, 
  departure_airport, 
  departure_schedule, 
  arrival_airport, 
  arrival_delay
FROM 
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  RAND() < 0.8
'''
df = pd.read_gbq(sql, project_id=project_id, dialect='standard')

print(tabulate(df, headers='keys', tablefmt='psql'))

KeyboardInterrupt: ignored

In [9]:
# Display query output immediately

%%bigquery --project evanproject-311200
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.48s/rows]


,total_rows
0,114420316


In [23]:
%%bigquery --project evanproject-311200
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.46s/rows]


,total_rows
0,114420316


In [15]:
from google.cloud import bigquery

client = bigquery.Client(project="evanproject-311200")

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


In [16]:
client = bigquery.Client(project="evanproject-311200")
temp = client.query('''
SELECT
  date, 
  airline, 
  departure_airport, 
  departure_schedule, 
  arrival_airport, 
  arrival_delay
FROM 
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  MOD(ABS(FARM_FINGERPRINT(date)), 10) IN (9)
  ''').to_dataframe()


In [21]:
print(temp.head())

         date airline  ... arrival_airport  arrival_delay
0  2006-08-17      KH  ...             KOA           -4.0
1  2006-08-14      KH  ...             KOA           -7.0
2  2006-08-11      KH  ...             KOA            0.0
3  2006-08-27      KH  ...             KOA            4.0
4  2007-11-15      KH  ...             KOA           -9.0

[5 rows x 6 columns]
